# General Neural Network

In [1]:
source = "GNN";

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
device = gpu;

## Data preparation

In [5]:
function get_data()
    # inputs are the user's ratings for all shows (unseen shows get mapped to zero) + implicit ratings + heterogenous features
    # outputs are the user's ratings for all shows (unseen shows get mapped to zero)
    training = get_residuals("recommendee", read_params(source)["residual_alphas"])
    implicit = get_implicit_list()
    X1 = zeros(Float32, num_items() + 1)
    X1[training.item] = training.rating
    X2 = zeros(Float32, num_items() + 1)
    X2[implicit.item] = implicit.rating

    count = convert(Float32, length(implicit.item) / num_items())
    X3 = [count, sqrt(count), count^2]
    X = vcat(X1, X2, X3)
end;

In [6]:
# custom split layer
struct Split{T}
    paths::T
end
Split(paths...) = Split(paths)
Flux.@functor Split
(m::Split)(x::AbstractArray) = map(f -> f(x), m.paths)

In [7]:
function compute_alpha()
    params = read_params(source)
    BSON.@load params["model"] m
    ratings, implicit = m(get_data())
    softmax = exp.(implicit)
    softmax = softmax ./ sum(softmax)
    write_recommendee_alpha(x -> ratings[x])
    write_recommendee_alpha(x -> softmax[x], outdir = "GNN_implicit")
end;

In [8]:
compute_alpha()